<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/aperiodic_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import random

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    """Generates Poisson-distributed flash onset times ensuring correct flash count and duty cycle adjustments."""
    expected_flashes = int(round(duration * frequency))  # Ensuring approximate expected count
    cycle_length = 1.0 / frequency  # Time per cycle
    times = []
    time = 0

    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)  # Allow more random variation
        interval = min(interval, cycle_length * max_interval_factor)  # Cap extreme Poisson outliers
        interval = max(interval, min_interval)  # Ensure minimum spacing

        on_duration = interval * (duty_cycle / 100)  # Adjust based on duty cycle
        time += interval
        if time < duration:
            times.append((time, on_duration))  # Store onset time and duration

    return times

def regularise_strobe(signal, min_interval=0.025):
    """Regularizes strobe sequence by handling overlaps but maintaining frequency integrity."""
    output_signal = []
    last_flash_end = -np.inf
    for onset, duration in signal:
        if onset >= last_flash_end + min_interval:
            output_signal.append((onset, duration))
            last_flash_end = onset + duration

    return output_signal

def calculate_effective_frequency(signal, duration):
    """Computes the effective frequency based on the number of flashes in the signal."""
    return len(signal) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000):
    """Finds the optimal initial frequency that produces the desired effective frequency."""
    F = target_Fe * 1.2  # Start slightly higher to account for Poisson spread
    iteration = 0

    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        flash_times = regularise_strobe(flash_times)
        Fe = calculate_effective_frequency(flash_times, duration)

        if abs(Fe - target_Fe) <= tolerance:
            return F, Fe, flash_times  # Found best match

        F = F + 0.15 if Fe < target_Fe else F - 0.15  # Adjust frequency more aggressively
        iteration += 1

    print("Warning: Max iterations reached without finding precise Fe match.")
    return F, Fe, flash_times

def generate_strobe_sequence():
    """Interactive function to generate a strobe sequence file, ensuring correct flash count."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()
    led_config = [input(f"LED Set {i+1} (1=On, 0=Off): ") for i in range(4)]

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']
    previous_onset = 0
    min_interval = 0.025
    num_steps = int(round(duration * (start_Fe + end_Fe) / 2))

    for i in range(num_steps):
        current_Fe = start_Fe + (end_Fe - start_Fe) * (i / (num_steps - 1))
        current_d = start_d + (end_d - start_d) * (i / (num_steps - 1))
        current_l = start_l + (end_l - start_l) * (i / (num_steps - 1))

        if stimulation.lower() == "aperiodic":
            # Optimize frequency per step to maintain correct effective frequency
            F, Fe, flash_times = optimize_initial_frequency(current_Fe, duration / num_steps, current_d)
        else:
            # Generate periodic sequence
            F = current_Fe
            cycle_length = 1.0 / F
            flash_times = [(j * cycle_length, cycle_length * (current_d / 100)) for j in range(int((duration / num_steps) * F))]

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)  # Ensure valid timing
            previous_onset = onset + on_duration

            # Assign first step to start frequency, last step to end frequency
            step_freq = start_Fe if i == 0 else end_Fe if i == num_steps - 1 else F

            step_string = f'{step_duration:.3f},1,{step_freq:.2f},{step_freq:.2f},{int(current_d)},{int(current_d)},{" ,".join(led_config)},{int(current_l)},{int(current_l)}'
            output.append(f'STP"{step_string}"')

    with open("strobe_sequence.txt", "w") as f:
        f.write('\n'.join(output))

    print("Generated sequence saved as strobe_sequence.txt")

# Run the generator
generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Aperiodic
Enter total sequence duration (seconds): 2
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
LED Set 1 (1=On, 0=Off): 1
LED Set 2 (1=On, 0=Off): 1
LED Set 3 (1=On, 0=Off): 1
LED Set 4 (1=On, 0=Off): 1
Generated sequence saved as strobe_sequence.txt
